### Installs

In [2]:
!pip install boto3

   ---------------------------------------- 0.0/139.2 kB ? eta -:--:--
   ---------------------------------------- 139.2/139.2 kB 4.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.4 MB ? eta -:--:--
    --------------------------------------- 0.2/12.4 MB 5.0 MB/s eta 0:00:03
   - -------------------------------------- 0.4/12.4 MB 4.6 MB/s eta 0:00:03
   -- ------------------------------------- 0.7/12.4 MB 5.0 MB/s eta 0:00:03
   --- ------------------------------------ 1.0/12.4 MB 5.2 MB/s eta 0:00:03
   ---- ----------------------------------- 1.3/12.4 MB 5.7 MB/s eta 0:00:02
   ----- ---------------------------------- 1.6/12.4 MB 5.7 MB/s eta 0:00:02
   ------ --------------------------------- 2.0/12.4 MB 5.9 MB/s eta 0:00:02
   ------- -------------------------------- 2.2/12.4 MB 5.9 MB/s eta 0:00:02
   ------- -------------------------------- 2.4/12.4 MB 5.8 MB/s eta 0:00:02
   --------- ------------------------------ 2.8/12.4 MB 6.0 MB/s eta 0:00:02
   ------


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: C:\Users\chafu\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
!pip install botocore


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: C:\Users\chafu\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [7]:
!pip install response


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: C:\Users\chafu\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


### Exploration

In [10]:
import boto3
import botocore.config
import json
from datetime import datetime

In [ ]:
import boto3
import botocore.config
import json
from datetime import datetime

def using_bedrock(message:str, categories:list)-> str:

    prompt=f"""<s>[INST]Human: Considering the following categories options
    {categories}, categorize the following message: {message}.
    Assistant:[/INST]
    """

    body={
        "prompt": prompt,
        "max_gen_len": 512,
        "temperature": 0.5,
        "top_p": 0.9,
    }

    try:
        bedrock=boto3.client("bedrock-runtime",
                             region_name="us-east-1",
                             config=botocore.config.Config(read_timeout=300,
                                                           retries={
                                                               'max_attempts':3
                                                               }))
        response=bedrock.invoke_model(body=json.dumps(body),
                             modelId="meta.llama3-70b-instruct-v1:0")
        
        response_content=response.get('body').read()
        response_data = json.loads(response_content.decode("utf-8"))
        print(response_data)

        response_detail = response_data['generation']
        
        return response_detail
    
    except Exception as e:
        print(f"Error generating the response: {e}")
        return "~"

def save_message_details_to_s3(s3_key, s3_bucket, original_message, classified_message):
    
    s3=boto3.client('s3')
    current_time = datetime.now().strftime('%Y-%m-%d_%H%M%S')
    message_to_save = f"{current_time}\nOriginal Message: {original_message}\nClassified Message: {classified_message}"

    try:
        s3.put_object(Bucket=s3_bucket, Key=s3_key, Body=message_to_save)
        print("Message saved to s3")
    
    except Exception as e:
        print(f"Error while saving the message to s3: {e}")

def lambda_handler(event, context):
    # TODO implement
    event=json.loads(event['body'])
    message_categ=event['message']
    categories = event['categories']
    generate_message=using_bedrock(message=message_categ, categories=categories)

    if generate_message:
        current_time=datetime.now().strftime('%Y-%m-%d_%H%M%S')
        s3_key=f"message-output/{current_time}.txt"
        s3_bucket='guscontini-aws-bedrock'
        save_message_details_to_s3(s3_key, s3_bucket, message_categ, generate_message)
        response_body = json.dumps({'classified_message': generate_message})

    else:
        print("No message was generated")
        response_body = json.dumps({'error': 'No message was generated'})
    
    return {
        'statusCode': 200,
        'body': response_body
    }


In [16]:
import requests
import json

def test_message_classification(message, categories):
    # URL da sua API Gateway
    api_url = "https://iqzolzrrni.execute-api.us-east-1.amazonaws.com/dev/message-classification"

    # Prepare a payload para a requisição POST
    payload = {
        "message": message,
        "categories": categories
    }
    
    # Faça a requisição POST
    response = requests.post(api_url, json=payload)

    # Verifique o código de status da resposta
    if response.status_code == 200:
        print("Classificação de mensagem realizada com sucesso!")
        result = response.json()
        print("Resposta da API:", result)
        
        # Extraia o conteúdo da resposta (opcional)
        message_content = result.get('body')  
        print("Conteúdo da mensagem:", message_content)

    else:
        print("Erro na classificação de mensagem:", response.status_code)
        print("Resposta da API:", response.text)  # Imprima a resposta da API em caso de erro

if __name__ == "__main__":
    # Exemplo de uso
    test_message = "Joe Biden and Donald Trump are candidates to the most import office of the world"
    test_categories = ["Esporte", "Política", "Tecnologia"]

    test_message_classification(test_message, test_categories)


Classificação de mensagem realizada com sucesso!
Resposta da API: {'classified_message': ' Política'}
Conteúdo da mensagem: None
